In [1]:
pip install pandas numpy nltk scikit-learn xgboost datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
from datasets import load_dataset

# Load the Banking77 dataset
dataset = load_dataset("PolyAI/banking77",trust_remote_code=True)

# Access train and test splits
train_data = dataset['train']
test_data = dataset['test']

# Convert to pandas DataFrame
import pandas as pd
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Combine train and test for preprocessing
df = pd.concat([train_df, test_df], ignore_index=True)

# Rename columns for consistency
df.rename(columns={'text': 'Text', 'label': 'Label'}, inplace=True)

# Map numerical labels to category names
label_mapping = {i: label for i, label in enumerate(dataset['train'].features['label'].names)}
df['Label'] = df['Label'].map(label_mapping)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.78k [00:00<?, ?B/s]

banking77.py:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.89k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [16]:
train_df.shape


(10003, 2)

In [5]:
test_df.shape

(3080, 2)

In [17]:
df.shape

(13083, 3)

In [18]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word,pos ="v") for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing
df['Clean_Text'] = df['Text'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

# Fit and transform the clean text
X = vectorizer.fit_transform(df['Clean_Text'])


In [20]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['Label'])


In [11]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import xgboost as xgb

# Initialize XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:01:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [13]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on test data
y_pred = model.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


Accuracy: 0.8318685517768437
Classification Report:
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.95      0.93      0.94        44
                                activate_my_card       0.87      0.96      0.91        55
                                       age_limit       1.00      1.00      1.00        33
                         apple_pay_or_google_pay       1.00      0.94      0.97        36
                                     atm_support       0.87      1.00      0.93        20
                                automatic_top_up       0.93      0.96      0.95        27
         balance_not_updated_after_bank_transfer       0.69      0.72      0.71        50
balance_not_updated_after_cheque_or_cash_deposit       0.94      0.86      0.90        51
                         beneficiary_not_allowed       0.88      0.91      0.90        33
                                 cancel_transfe

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the already vectorized data (X) and target (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression model
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Train the model using the vectorized data
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.97      0.89      0.93        44
                                activate_my_card       0.98      0.96      0.97        55
                                       age_limit       0.97      0.97      0.97        33
                         apple_pay_or_google_pay       1.00      1.00      1.00        36
                                     atm_support       0.90      0.95      0.93        20
                                automatic_top_up       0.96      0.93      0.94        27
         balance_not_updated_after_bank_transfer       0.67      0.74      0.70        50
balance_not_updated_after_cheque_or_cash_deposit       0.87      0.80      0.84        51
                         beneficiary_not_allowed       0.82      0.97      0.89        33
                                 cancel_transfer       0.93      0.93      0.93        41
         

In [24]:
from google.colab import drive
import joblib

# Mount Google Drive
drive.mount('/content/drive')

# Save model
joblib.dump(pipeline, '/content/drive/MyDrive/BankingData/banking77_model_01.joblib')
print("✅ Model saved to your Google Drive.")

Mounted at /content/drive
✅ Model saved to your Google Drive.


In [25]:
import joblib

# Load the trained model
model = joblib.load('/content/drive/MyDrive/BankingData/banking77_model.joblib')

# Define a list of diverse banking-related queries
queries = [
    "I lost my card",
    "How can I get a new credit card?",
    "Can I increase my overdraft limit?",
    "I'd like to cancel a payment",
    "Tell me your working hours",
    "I want to activate my new card",
    "What's the exchange rate for USD to EUR?",
    "My card was declined at the ATM",
    "How do I change my account password?",
    "I need to verify my identity",
    "Why was I charged twice for the same transaction?",
    "Is Apple Pay supported?",
    "I forgot my PIN",
    "How do I close my account?",
    "My refund hasn't appeared yet",
    "Can I get a virtual card?",
    "What's the limit for cash withdrawals?",
    "I want to link another bank account",
    "My card is about to expire",
    "How do I set up automatic top-up?"
]

# Iterate through the queries and predict intents
for query in queries:
    predicted_intent = model.predict([query])[0]
    print(f"❓ Query: {query}\n➡️ Predicted intent: {predicted_intent}\n")



❓ Query: I lost my card
➡️ Predicted intent: lost_or_stolen_card

❓ Query: How can I get a new credit card?
➡️ Predicted intent: card_arrival

❓ Query: Can I increase my overdraft limit?
➡️ Predicted intent: top_up_limits

❓ Query: I'd like to cancel a payment
➡️ Predicted intent: cancel_transfer

❓ Query: Tell me your working hours
➡️ Predicted intent: card_not_working

❓ Query: I want to activate my new card
➡️ Predicted intent: activate_my_card

❓ Query: What's the exchange rate for USD to EUR?
➡️ Predicted intent: exchange_rate

❓ Query: My card was declined at the ATM
➡️ Predicted intent: declined_cash_withdrawal

❓ Query: How do I change my account password?
➡️ Predicted intent: passcode_forgotten

❓ Query: I need to verify my identity
➡️ Predicted intent: verify_my_identity

❓ Query: Why was I charged twice for the same transaction?
➡️ Predicted intent: transaction_charged_twice

❓ Query: Is Apple Pay supported?
➡️ Predicted intent: apple_pay_or_google_pay

❓ Query: I forgot my 